In [4]:
import pandas as pd
import minsearch as minsearch
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
from openai import OpenAI
from tqdm.auto import tqdm

# Retrieval Evaluation

In [5]:
df = pd.read_csv("../data/stoic_zen_document.csv")
df.insert(0,'id',df.index)

documents = df.to_dict(orient="records")
print("lenght of the documents:", len(documents))

index = minsearch.Index(
    text_fields=["category", "question", "answer"],
    keyword_fields=['id',"ideology"]
)

index.fit(documents)

lenght of the documents: 820


In [6]:
df_questions = pd.read_csv("../data/ground_truth_retrieval.csv")
ground_truth = df_questions.to_dict(orient='records')

In [7]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def minsearch_search(query, ideology):
    boost = {'question': 3.0, 'category': 0.5}

    results = index.search(
        query=query,
        filter_dict={'ideology': ideology},
        boost_dict=boost,
        num_results=5
    )

    return results

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [6]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'],q['ideology']))

  0%|          | 0/4100 [00:00<?, ?it/s]

{'hit_rate': 0.8043902439024391, 'mrr': 0.6790121951219511}

# Finding the best parameters

In [32]:
df_validation = df_questions[:200]
gt_val = df_validation.to_dict(orient='records')

df_test = df_questions[200:]

In [33]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [41]:
def minsearch_search(query, ideology, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={'ideology': ideology},
        boost_dict=boost,
        num_results=5
    )

    return results

In [42]:
param_ranges = {
    'question': (0.0, 5.0),
    'category': (0.0, 5.0)
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], q['ideology'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [43]:
def minsearch_improved(query,ideology):
    boost = {
        'question': 1.22,
        'category': 0.36
    }

    results = index.search(
        query=query,
        filter_dict={'ideology': ideology},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['question'], q['ideology']))

  0%|          | 0/4100 [00:00<?, ?it/s]

{'hit_rate': 0.9185365853658537, 'mrr': 0.7592213511420829}